In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pickle
import PyPDF2


In [ ]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("/content/UpdatedResumeDataSet.csv")

In [ ]:
def cleanResume(txt):
    cleanText = re.sub('http\S+\s', ' ', txt)
    cleanText = re.sub('RT|cc', ' ', cleanText)
    cleanText = re.sub('#\S+\s', ' ', cleanText)
    cleanText = re.sub('@\S+', '  ', cleanText)
    cleanText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', cleanText)
    cleanText = re.sub(r'[^\x00-\x7f]', ' ', cleanText)
    cleanText = re.sub('\s+', ' ', cleanText)
    return cleanText

In [ ]:
df.head()


,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [ ]:
df['Resume'] = df['Resume'].apply(lambda x: cleanResume(x))


In [ ]:
le = LabelEncoder()
le.fit(df['Category'])
df['Category'] = le.transform(df['Category'])

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')


In [ ]:
tfidf.fit(df['Resume'])
requiredText = tfidf.transform(df['Resume'])

In [ ]:
  X_train, X_test, y_train, y_test = train_test_split(requiredText, df['Category'], test_size=0.2, random_state=42)



In [ ]:
clf = OneVsRestClassifier(SVC(kernel='linear', C=1.0, probability=True, random_state=42))


In [ ]:
clf.fit(X_train, y_train)


OneVsRestClassifier(estimator=SVC(kernel='linear', probability=True,
                                  random_state=42))

In [ ]:
ypred = clf.predict(X_test)


In [ ]:
accuracy = accuracy_score(y_test, ypred)
print("Accuracy:", accuracy)

Accuracy: 0.9948186528497409


In [ ]:
pickle.dump(tfidf, open('tfidf.pkl', 'wb'))
pickle.dump(clf, open('svm_classifier.pkl', 'wb'))

In [ ]:
pdf_resume_path = "E:\MCA\mini project\sample resume.pdf"


In [ ]:
def extract_text_from_pdf(pdf_path):
    pdf_text = ""
    try:
        with open(pdf_path, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfFileReader(pdf_file)
            for page_num in range(pdf_reader.numPages):
                page = pdf_reader.getPage(page_num)
                pdf_text += page.extractText()
    except Exception as e:
        return str(e)
    return pdf_text

In [ ]:
pdf_resume_path = "E:\MCA\mini project\Java-developer-Resume-Sample.pdf"


In [ ]:
pdf_resume_text = extract_text_from_pdf(pdf_resume_path)

In [ ]:
cleaned_resume = cleanResume(pdf_resume_text)

In [ ]:
input_features = tfidf.transform([cleaned_resume])


In [ ]:
prediction_id = clf.predict(input_features)[0]

In [ ]:
category_mapping= {
    # Define your category mapping here
    15: "Java Developer",
    23: "Testing",
    8: "DevOps Engineer",
    20: "Python Developer",
    24: "Web Designing",
    12: "HR",
    13: "Hadoop",
    3: "Blockchain",
    10: "ETL Developer",
    18: "Operations Manager",
    6: "Data Science",
    22: "Sales",
    16: "Mechanical Engineer",
    1: "Arts",
    7: "Database",
    11: "Electrical Engineering",
    14: "Health and fitness",
    19: "PMO",
    4: "Business Analyst",
    9: "DotNet Developer",
    2: "Automation Testing",
    17: "Network Security Engineer",
    21: "SAP Developer",
    5: "Civil Engineer",
    0: "Advocate",
}


In [ ]:
category_name = category_mapping.get(prediction_id, "Unknown")


In [ ]:
print("Predicted Category:", category_name)


Predicted Category: Java Developer


In [ ]:
import joblib
modelfile="tfidf.pkl"
joblib.dump(tfidf,modelfile)
from google.colab import files
files.download(modelfile)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import joblib
modelfile="svm_classifier.pkl"
joblib.dump(clf,modelfile)
from google.colab import files
files.download(modelfile)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>